This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# xarray furuno backend

In this example, we read scn/scnx (furuno) data files using the wradlib `furuno` xarray backend.

Furuno Weather Radars generate binary files. The binary version depend on the radar type. This reader is able to consume SCN (format version 3) and SCNX (format version 10) files. 

Uncompressed files are read via ``numpy.memmap`` with lazy-loading mechanism. Gzip compressed files are are opened, read into memory and processed using ``numpy.frombuffer``.

Radar moments are read as packed data with 16-bit resolution and output as 32bit-floating point data.

In [ ]:
import glob
import gzip
import io
import wradlib as wrl
import warnings

warnings.filterwarnings("ignore")
import matplotlib.pyplot as pl
import numpy as np
import xarray as xr

try:
    get_ipython().run_line_magic("matplotlib inline")
except:
    pl.ion()

## Load furuno scn Data

Data provided by University of Graz, Austria.

In [ ]:
fpath = "furuno/0080_20210730_160000_01_02.scn.gz"
f = wrl.util.get_wradlib_data_file(fpath)
vol = wrl.io.open_furuno_dataset(f, reindex_angle=False)

In [ ]:
vol[0]

### Inspect RadarVolume

In [ ]:
display(vol)

### Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

### Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol[0])

### Goereferencing

In [ ]:
swp = vol[0].copy().pipe(wrl.georef.georeference_dataset)

### Plotting

Currently the data dynamic range is left as read from the file. That way the difference between shortpulse and longpulse can be clearly seen.

In [ ]:
swp.DBZH.plot.pcolormesh(x="x", y="y")
pl.gca().set_aspect("equal")

In [ ]:
fig = pl.figure(figsize=(10, 10))
swp.DBZH.wradlib.plot_ppi(proj="cg", fig=fig)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
pm = swp.DBZH.wradlib.plot_ppi(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
import cartopy.feature as cfeature


def plot_rivers(ax):
    rivers = cfeature.NaturalEarthFeature(
        category="physical",
        name="rivers_lake_centerlines",
        scale="10m",
        facecolor="none",
    )
    ax.add_feature(rivers, edgecolor="blue", lw=2, zorder=4)


map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)

DBZH = swp.DBZH
pm = DBZH.where(DBZH > 0).wradlib.plot_ppi(ax=ax)
plot_rivers(ax)
ax.gridlines(draw_labels=True)

In [ ]:
import matplotlib.path as mpath

theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values,
    central_longitude=swp.longitude.values,
)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
ax.set_boundary(circle, transform=ax.transAxes)

pm = swp.DBZH.wradlib.plot_ppi(proj=map_proj, ax=ax)
ax = pl.gca()
ax.gridlines(crs=map_proj)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines()

In [ ]:
swp.DBZH.wradlib.plot_ppi()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset.

In [ ]:
display(swp.DBZH)

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('rtime')` method, which sorts the radials by time.

In [ ]:
swp.DBZH.sortby("rtime").plot(x="range", y="rtime", add_labels=False)

In [ ]:
fig = pl.figure(figsize=(5, 5))
pm = swp.DBZH.wradlib.plot_ppi(proj={"latmin": 3e3}, fig=fig)

### Mask some values

In [ ]:
dbzh = swp["DBZH"].where(swp["DBZH"] >= 0)
dbzh.plot(x="x", y="y")

In [ ]:
vol[0]

### Export to ODIM and CfRadial2


In [ ]:
vol[0].DBZH.sortby("rtime").plot(y="rtime")

In [ ]:
vol.to_odim("furuno_scn_as_odim.h5")
vol.to_cfradial2("furuno_scn_as_cfradial2.nc")

### Import again

In [ ]:
vola = wrl.io.open_odim_dataset(
    "furuno_scn_as_odim.h5", reindex_angle=False, keep_elevation=True
)
display(vola.root)
display(vola[0])
vola[0].DBZH.sortby("rtime").plot(y="rtime")

In [ ]:
volb = wrl.io.open_cfradial2_dataset("furuno_scn_as_cfradial2.nc")
display(volb.root)
display(volb[0])
volb[0].DBZH.sortby("rtime").plot(y="rtime")

### Check equality

There are slight differences due to importing/exporting and calculation with floating point data.

In [ ]:
xr.testing.assert_allclose(vol.root, vola.root)
xr.testing.assert_allclose(
    vol[0].time.astype(float).reset_coords(drop=True),
    vola[0].time.astype(float).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
    vola[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
    vola[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
)


xr.testing.assert_allclose(vol.root, volb.root)
xr.testing.assert_allclose(
    vol[0].time.astype(float).reset_coords(drop=True),
    volb[0].time.astype(float).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
)

xr.testing.assert_allclose(vola.root, volb.root)
xr.testing.assert_allclose(
    vola[0].time.astype(float).reset_coords(drop=True),
    volb[0].time.astype(float).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vola[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vola[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
)

### Use `xr.open_dataset` to retrieve explicit group

<div class="alert alert-warning">

**Warning** <br>
    
Since $\omega radlib$ version 1.18 the xarray backend engines for polar radar data have been renamed and prepended with `wradlib-` (eg. `furuno` -> `wradlib-furuno`). This was necessary to avoid clashes with the new [xradar](https://github.com/openradar/xradar)-package, which will eventually replace the wradlib engines. Users have to make sure to check which engine to use for their use-case when using `xarray.open_dataset`. Users might install and test `xradar`, and check if it is already robust enough for their use-cases (by using xradar's `engine="furuno"`.
    
Since $\omega radlib$ version 1.19 the xarray backend engines for polar radar data have been deprecated. The functionality is kept until wradlib version 2.0, when the backend-code will be removed completely. wradlib is importing that functionality from [xradar](https://github.com/openradar/xradar)-package whenever and wherever necessary.

Below we use a compatibility layer in wradlib to give users the chance to adapt their code. The first minimal change is that for every backend the group-layout is conforming to the CfRadial-standard naming scheme (``sweep_0``, ``sweep_1``, etc.).
    
</div>

In [ ]:
swp = xr.open_dataset(
    f,
    engine="wradlib-furuno",
    group="sweep_0",
    backend_kwargs=dict(reindex_angle=False),
)
display(swp)

## Load furuno scnx Data

Data provided by GFZ German Research Centre for Geosciences.

In [ ]:
fpath = "furuno/2006_20220324_000000_000.scnx.gz"
f = wrl.util.get_wradlib_data_file(fpath)
vol = wrl.io.open_furuno_dataset(f, reindex_angle=False)

### Inspect RadarVolume

In [ ]:
display(vol)

### Inspect root group

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
vol.root

### Inspect sweep group(s)

The sweep-groups can be accessed via their respective keys. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
display(vol[0])

### Goereferencing

In [ ]:
swp = vol[0].copy().pipe(wrl.georef.georeference_dataset)

### Plotting

Currently the data dynamic range is left as read from the file. That way the difference between shortpulse and longpulse can be clearly seen.

In [ ]:
swp.DBZH.plot.pcolormesh(x="x", y="y")
pl.gca().set_aspect("equal")

In [ ]:
fig = pl.figure(figsize=(10, 10))
swp.DBZH.wradlib.plot_ppi(proj="cg", fig=fig)

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

map_trans = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)

In [ ]:
map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
pm = swp.DBZH.wradlib.plot_ppi(proj=map_proj)
ax = pl.gca()
ax.gridlines(crs=map_proj)
print(ax)

In [ ]:
map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
pm = swp.DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines(draw_labels=True)

In [ ]:
import cartopy.feature as cfeature


def plot_rivers(ax):
    rivers = cfeature.NaturalEarthFeature(
        category="physical",
        name="rivers_lake_centerlines",
        scale="10m",
        facecolor="none",
    )
    ax.add_feature(rivers, edgecolor="blue", lw=2, zorder=4)


map_proj = ccrs.Mercator(central_longitude=swp.longitude.values)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)

DBZH = swp.DBZH
pm = DBZH.where(DBZH > 0).wradlib.plot_ppi(ax=ax)
plot_rivers(ax)
ax.gridlines(draw_labels=True)

In [ ]:
import matplotlib.path as mpath

theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

map_proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values,
    central_longitude=swp.longitude.values,
)
fig = pl.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=map_proj)
ax.set_boundary(circle, transform=ax.transAxes)

pm = swp.DBZH.wradlib.plot_ppi(proj=map_proj, ax=ax)
ax = pl.gca()
ax.gridlines(crs=map_proj)

In [ ]:
fig = pl.figure(figsize=(10, 8))
proj = ccrs.AzimuthalEquidistant(
    central_latitude=swp.latitude.values, central_longitude=swp.longitude.values
)
ax = fig.add_subplot(111, projection=proj)
pm = swp.DBZH.wradlib.plot_ppi(ax=ax)
ax.gridlines()

In [ ]:
swp.DBZH.wradlib.plot_ppi()

### Inspect radar moments

The DataArrays can be accessed by key or by attribute. Each DataArray has dimensions and coordinates of it's parent dataset.

In [ ]:
display(swp.DBZH)

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('rtime')` method, which sorts the radials by time.

In [ ]:
swp.DBZH.sortby("rtime").plot(x="range", y="rtime", add_labels=False)

In [ ]:
fig = pl.figure(figsize=(5, 5))
pm = swp.DBZH.wradlib.plot_ppi(proj={"latmin": 3e3}, fig=fig)

### Mask some values

In [ ]:
dbzh = swp["DBZH"].where(swp["DBZH"] >= 0)
dbzh.plot(x="x", y="y")

In [ ]:
vol[0]

### Export to ODIM and CfRadial2


In [ ]:
vol[0].DBZH.sortby("rtime").plot(y="rtime")

In [ ]:
vol.to_odim("furuno_scnx_as_odim.h5")
vol.to_cfradial2("furuno_scnx_as_cfradial2.nc")

### Import again

In [ ]:
vola = wrl.io.open_odim_dataset(
    "furuno_scnx_as_odim.h5", reindex_angle=False, keep_elevation=True
)
display(vola.root)
display(vola[0])
vola[0].DBZH.sortby("rtime").plot(y="rtime")

In [ ]:
volb = wrl.io.open_cfradial2_dataset("furuno_scnx_as_cfradial2.nc")
display(volb.root)
display(volb[0])
volb[0].DBZH.sortby("rtime").plot(y="rtime")

### Check equality

There are slight differences due to importing/exporting and calculation with floating point data.

In [ ]:
xr.testing.assert_allclose(vol.root, vola.root)
xr.testing.assert_allclose(
    vol[0].time.astype(float).reset_coords(drop=True),
    vola[0].time.astype(float).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
    vola[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
    vola[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
)


xr.testing.assert_allclose(vol.root, volb.root)
xr.testing.assert_allclose(
    vol[0].time.astype(float).reset_coords(drop=True),
    volb[0].time.astype(float).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vol[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
)

xr.testing.assert_allclose(vola.root, volb.root)
xr.testing.assert_allclose(
    vola[0].time.astype(float).reset_coords(drop=True),
    volb[0].time.astype(float).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vola[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime", "azimuth"]).reset_coords(drop=True),
)
xr.testing.assert_allclose(
    vola[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
    volb[0].drop_vars(["time", "rtime"]).azimuth.reset_coords(drop=True),
)

## More Furuno loading mechanisms

### Use `xr.open_dataset` to retrieve explicit group

<div class="alert alert-warning">

**Warning** <br>
    
Since $\omega radlib$ version 1.18 the xarray backend engines for polar radar data have been renamed and prepended with `wradlib-` (eg. `furuno` -> `wradlib-furuno`). This was necessary to avoid clashes with the new [xradar](https://github.com/openradar/xradar)-package, which will eventually replace the wradlib engines. Users have to make sure to check which engine to use for their use-case when using `xarray.open_dataset`. Users might install and test `xradar`, and check if it is already robust enough for their use-cases (by using xradar's `engine="furuno"`.
    
Since $\omega radlib$ version 1.19 the xarray backend engines for polar radar data have been deprecated. The functionality is kept until wradlib version 2.0, when the backend-code will be removed completely. wradlib is importing that functionality from [xradar](https://github.com/openradar/xradar)-package whenever and wherever necessary.

Below we use a compatibility layer in wradlib to give users the chance to adapt their code. The first minimal change is that for every backend the group-layout is conforming to the CfRadial-standard naming scheme (``sweep_0``, ``sweep_1``, etc.).
    
</div>

Below you can inspect the main differences of the wradlib compatibility layer and the plain xradar implementation.

### use wradlib compatibility layer

In [ ]:
swp_a = xr.open_dataset(
    f, engine="wradlib-furuno", backend_kwargs=dict(reindex_angle=False)
)
display(swp_a)

### use xradar backend

In [ ]:
swp_b = xr.open_dataset(f, engine="furuno", backend_kwargs=dict(reindex_angle=False))
display(swp_b)